# Futures Data Visualization Analysis

Analysis of LME, SHFE, and CMX copper futures data with beautiful visualizations.

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import pyodbc
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.gridspec import GridSpec
import matplotlib.font_manager as fm
import seaborn as sns
import warnings

# Add project root to Python path
project_root = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
sys.path.insert(0, project_root)

from config.database_config import get_connection_string

warnings.filterwarnings('ignore')

# Set font to avoid Japanese character issues
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# Style settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['figure.titlesize'] = 16
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 150
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

# Color palette
COLORS = {
    'primary': '#2E86AB',
    'secondary': '#A23B72',
    'accent': '#F18F01',
    'success': '#C73E1D',
    'dark': '#2D3436',
    'light': '#F7F7F7'
}

# Gradient colors
gradient_colors = ['#667eea', '#764ba2', '#f093fb', '#f5576c', '#4facfe', '#00f2fe']

print("Libraries imported successfully")

## Data Connection Test

In [ ]:
# Test database connection
try:
    conn = pyodbc.connect(get_connection_string())
    print("Database connected successfully")
    
    # Test query to check data availability
    test_query = """
    SELECT TOP 10 
        p.TradeDate,
        m.MetalCode,
        m.ExchangeCode,
        t.TenorTypeName,
        p.SettlementPrice
    FROM T_CommodityPrice p
    INNER JOIN M_Metal m ON p.MetalID = m.MetalID
    INNER JOIN M_TenorType t ON p.TenorTypeID = t.TenorTypeID
    WHERE p.SettlementPrice IS NOT NULL
    ORDER BY p.TradeDate DESC
    """
    
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", message="pandas only supports SQLAlchemy")
        test_df = pd.read_sql(test_query, conn)
    
    print(f"\nTest query returned {len(test_df)} rows")
    print("\nSample data:")
    print(test_df)
    
except Exception as e:
    print(f"Error: {e}")

## Data Retrieval Function

In [ ]:
def get_futures_data(conn, days=90):
    """Retrieve futures data"""
    query = f"""
    SELECT 
        p.TradeDate,
        m.MetalCode,
        m.ExchangeCode,
        t.TenorTypeName,
        p.SettlementPrice,
        p.Volume,
        p.OpenInterest,
        CASE 
            WHEN t.TenorTypeName LIKE 'Generic 1%' THEN 1
            WHEN t.TenorTypeName LIKE 'Generic 2%' THEN 2
            WHEN t.TenorTypeName LIKE 'Generic 3%' THEN 3
            WHEN t.TenorTypeName LIKE 'Generic 4%' THEN 4
            WHEN t.TenorTypeName LIKE 'Generic 5%' THEN 5
            WHEN t.TenorTypeName LIKE 'Generic 6%' THEN 6
            WHEN t.TenorTypeName LIKE 'Generic 7%' THEN 7
            WHEN t.TenorTypeName LIKE 'Generic 8%' THEN 8
            WHEN t.TenorTypeName LIKE 'Generic 9%' THEN 9
            WHEN t.TenorTypeName LIKE 'Generic 10%' THEN 10
            WHEN t.TenorTypeName LIKE 'Generic 11%' THEN 11
            WHEN t.TenorTypeName LIKE 'Generic 12%' THEN 12
            ELSE 0
        END as TenorNumber
    FROM T_CommodityPrice p
    INNER JOIN M_Metal m ON p.MetalID = m.MetalID
    INNER JOIN M_TenorType t ON p.TenorTypeID = t.TenorTypeID
    WHERE 
        t.TenorTypeName LIKE 'Generic%Future%'
        AND p.TradeDate >= DATEADD(day, -{days}, GETDATE())
        AND p.SettlementPrice IS NOT NULL
    ORDER BY p.TradeDate DESC, m.ExchangeCode, t.TenorTypeID
    """
    
    print(f"Executing query for last {days} days...")
    
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", message="pandas only supports SQLAlchemy")
        df = pd.read_sql(query, conn)
    
    df['TradeDate'] = pd.to_datetime(df['TradeDate'])
    return df

# Get data
futures_df = get_futures_data(conn, days=90)
print(f"\nRetrieved {len(futures_df):,} records")

if len(futures_df) > 0:
    print(f"Date range: {futures_df['TradeDate'].min().strftime('%Y-%m-%d')} to {futures_df['TradeDate'].max().strftime('%Y-%m-%d')}")
    
    # Check exchanges
    exchanges = futures_df['ExchangeCode'].dropna().unique()
    print(f"Exchanges: {', '.join(exchanges)}")
    
    # Check data by exchange
    print("\nData count by exchange:")
    print(futures_df.groupby('ExchangeCode').size())
else:
    print("No data found! Please check if data exists in the database.")

## 1. Futures Curve - Latest Trading Day

In [ ]:
if len(futures_df) > 0:
    # Filter LME data
    lme_data = futures_df[futures_df['ExchangeCode'] == 'LME'].copy()
    
    if len(lme_data) > 0:
        # Get latest trading day
        latest_date = lme_data['TradeDate'].max()
        latest_lme = lme_data[lme_data['TradeDate'] == latest_date].copy()
        
        # Sort by tenor
        latest_lme = latest_lme.sort_values('TenorNumber')
        
        # Create plot
        fig, ax = plt.subplots(figsize=(12, 6))
        
        # Plot futures curve
        x = latest_lme['TenorNumber']
        y = latest_lme['SettlementPrice']
        
        ax.fill_between(x, y, alpha=0.3, color=COLORS['primary'])
        ax.plot(x, y, color=COLORS['primary'], linewidth=3, marker='o', 
                markersize=8, markerfacecolor='white', markeredgewidth=2)
        
        # Add value labels
        for i, (tenor, price) in enumerate(zip(x, y)):
            if i % 2 == 0:  # Show every other label
                ax.annotate(f'${price:,.0f}', (tenor, price), 
                           textcoords="offset points", xytext=(0,10), 
                           ha='center', fontsize=9, weight='bold')
        
        ax.set_title(f'LME Copper Futures Curve ({latest_date.strftime("%Y-%m-%d")})',
                    fontsize=16, weight='bold', pad=20)
        ax.set_xlabel('Contract Month', fontsize=12)
        ax.set_ylabel('Price (USD/t)', fontsize=12)
        ax.set_xticks(x)
        ax.set_xticklabels([f'M{i}' for i in x])
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Summary statistics
        print(f"\nLME Futures Curve Summary:")
        print(f"Front month (M1): ${latest_lme[latest_lme['TenorNumber']==1]['SettlementPrice'].values[0]:,.2f}")
        print(f"Average price: ${y.mean():,.2f}")
        print(f"Price range: ${y.min():,.2f} - ${y.max():,.2f}")
        
        # Calculate spreads
        if len(latest_lme) > 1:
            m1_price = latest_lme[latest_lme['TenorNumber']==1]['SettlementPrice'].values[0]
            m3_price = latest_lme[latest_lme['TenorNumber']==3]['SettlementPrice'].values[0] if 3 in latest_lme['TenorNumber'].values else None
            
            if m3_price:
                spread_1_3 = m3_price - m1_price
                print(f"M1-M3 spread: ${spread_1_3:,.2f} ({('Contango' if spread_1_3 > 0 else 'Backwardation')})")
    else:
        print("No LME data found")
else:
    print("No futures data available")

## 2. Price Comparison Across Exchanges

In [ ]:
if len(futures_df) > 0:
    # Filter for front month contracts
    first_month = futures_df[futures_df['TenorNumber'] == 1].copy()
    first_month = first_month.sort_values('TradeDate')
    
    if len(first_month) > 0:
        fig, ax = plt.subplots(figsize=(14, 7))
        
        # Plot each exchange
        for exchange, color in zip(['LME', 'SHFE', 'CMX'], 
                                  [COLORS['primary'], COLORS['secondary'], COLORS['accent']]):
            exchange_data = first_month[first_month['ExchangeCode'] == exchange]
            if not exchange_data.empty:
                ax.plot(exchange_data['TradeDate'], exchange_data['SettlementPrice'],
                       label=exchange, color=color, linewidth=2.5, alpha=0.8)
                
                # Mark latest price
                latest = exchange_data.iloc[-1]
                ax.scatter(latest['TradeDate'], latest['SettlementPrice'],
                          color=color, s=100, zorder=5)
                ax.annotate(f'{exchange}\n${latest["SettlementPrice"]:,.0f}',
                           xy=(latest['TradeDate'], latest['SettlementPrice']),
                           xytext=(10, 10), textcoords='offset points',
                           fontsize=10, color=color, weight='bold',
                           bbox=dict(boxstyle='round,pad=0.5', facecolor='white', 
                                    edgecolor=color, alpha=0.8))
        
        ax.set_title('Copper Futures Price Comparison - Front Month', fontsize=16, weight='bold')
        ax.set_xlabel('Trade Date', fontsize=12)
        ax.set_ylabel('Price (USD/t)', fontsize=12)
        ax.legend(loc='best', frameon=True, fancybox=True, shadow=True)
        ax.grid(True, alpha=0.3)
        
        # Format dates
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
        ax.xaxis.set_major_locator(mdates.DayLocator(interval=7))
        plt.xticks(rotation=45)
        
        plt.tight_layout()
        plt.show()
        
        # Price comparison summary
        print("\nLatest Front Month Prices:")
        for exchange in ['LME', 'SHFE', 'CMX']:
            exchange_data = first_month[first_month['ExchangeCode'] == exchange]
            if not exchange_data.empty:
                latest = exchange_data.iloc[-1]
                print(f"{exchange}: ${latest['SettlementPrice']:,.2f} ({latest['TradeDate'].strftime('%Y-%m-%d')})")
    else:
        print("No front month data found")
else:
    print("No data available")

## 3. Term Structure Heatmap

In [ ]:
if len(futures_df) > 0 and 'LME' in futures_df['ExchangeCode'].values:
    lme_data = futures_df[futures_df['ExchangeCode'] == 'LME'].copy()
    
    # Get last 20 trading days
    recent_dates = sorted(lme_data['TradeDate'].unique(), reverse=True)[:20]
    lme_recent = lme_data[lme_data['TradeDate'].isin(recent_dates)]
    
    # Create pivot table
    pivot_data = lme_recent.pivot_table(
        values='SettlementPrice',
        index='TradeDate',
        columns='TenorNumber',
        aggfunc='mean'
    ).sort_index(ascending=True)
    
    if not pivot_data.empty:
        # Create heatmap
        fig, ax = plt.subplots(figsize=(12, 8))
        
        # Use seaborn heatmap
        sns.heatmap(pivot_data, 
                   cmap='RdYlBu_r',
                   center=pivot_data.mean().mean(),
                   cbar_kws={'label': 'Price (USD/t)'},
                   fmt='.0f',
                   linewidths=0.5,
                   ax=ax)
        
        # Customize labels
        ax.set_xticklabels([f'M{int(x.get_text())}' for x in ax.get_xticklabels()])
        ax.set_yticklabels([pd.to_datetime(y.get_text()).strftime('%m/%d') for y in ax.get_yticklabels()])
        
        ax.set_title('LME Copper Futures Term Structure Heatmap', fontsize=16, weight='bold', pad=20)
        ax.set_xlabel('Contract Month', fontsize=12)
        ax.set_ylabel('Trade Date', fontsize=12)
        
        plt.tight_layout()
        plt.show()
        
        # Calculate average term structure slope
        latest_prices = pivot_data.iloc[-1].dropna()
        if len(latest_prices) > 1:
            slope = (latest_prices.iloc[-1] - latest_prices.iloc[0]) / (len(latest_prices) - 1)
            print(f"\nAverage term structure slope: ${slope:.2f} per month")
            print(f"Term structure: {'Contango' if slope > 0 else 'Backwardation'}")
    else:
        print("Not enough data for heatmap")
else:
    print("No LME data available for heatmap")

## 4. Volume and Open Interest Analysis

In [ ]:
if len(futures_df) > 0 and 'LME' in futures_df['ExchangeCode'].values:
    lme_data = futures_df[futures_df['ExchangeCode'] == 'LME'].copy()
    
    # Calculate average volume and OI by tenor
    lme_liquidity = lme_data.groupby('TenorNumber').agg({
        'Volume': 'mean',
        'OpenInterest': 'mean'
    }).round(0)
    
    if not lme_liquidity.empty:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
        
        # Volume chart
        tenors = lme_liquidity.index
        volumes = lme_liquidity['Volume']
        
        bars1 = ax1.bar(tenors, volumes, color=gradient_colors[0], alpha=0.8)
        
        # Add gradient effect
        for i, (bar, color) in enumerate(zip(bars1, gradient_colors[:len(bars1)])):
            if i < len(gradient_colors):
                bar.set_color(color)
                bar.set_alpha(0.8)
        
        ax1.set_title('Average Daily Volume by Contract Month', fontsize=14, weight='bold')
        ax1.set_xlabel('Contract Month', fontsize=12)
        ax1.set_ylabel('Volume (contracts)', fontsize=12)
        ax1.set_xticks(tenors)
        ax1.set_xticklabels([f'M{i}' for i in tenors])
        ax1.grid(True, alpha=0.3, axis='y')
        
        # Open Interest chart
        oi = lme_liquidity['OpenInterest']
        
        bars2 = ax2.bar(tenors, oi, color=gradient_colors[3], alpha=0.8)
        
        # Add gradient effect
        for i, (bar, color) in enumerate(zip(bars2, gradient_colors[3:])):
            if i < len(gradient_colors) - 3:
                bar.set_color(color)
                bar.set_alpha(0.8)
        
        ax2.set_title('Average Open Interest by Contract Month', fontsize=14, weight='bold')
        ax2.set_xlabel('Contract Month', fontsize=12)
        ax2.set_ylabel('Open Interest (contracts)', fontsize=12)
        ax2.set_xticks(tenors)
        ax2.set_xticklabels([f'M{i}' for i in tenors])
        ax2.grid(True, alpha=0.3, axis='y')
        
        plt.suptitle('LME Copper Futures Liquidity Analysis', fontsize=16, weight='bold')
        plt.tight_layout()
        plt.show()
        
        # Liquidity concentration
        if len(volumes) >= 3:
            total_volume = volumes.sum()
            total_oi = oi.sum()
            front_3_volume = volumes[:3].sum() / total_volume * 100 if total_volume > 0 else 0
            front_3_oi = oi[:3].sum() / total_oi * 100 if total_oi > 0 else 0
            
            print(f"\nLiquidity Analysis:")
            print(f"Front 3 months volume concentration: {front_3_volume:.1f}%")
            print(f"Front 3 months open interest concentration: {front_3_oi:.1f}%")
            print(f"Most liquid contract: M{volumes.idxmax()}")
    else:
        print("No liquidity data available")
else:
    print("No LME data available for liquidity analysis")

## Close Database Connection

In [ ]:
conn.close()
print("Database connection closed successfully")